In [1]:
# 爬小说
import re
import requests
import time
from multiprocessing.dummy import Pool


In [2]:
# 再次清空一下?号
def clearQ(s):
    l = len(s)
    re = ''
    for i in range(l):
        if s[i]=='?':
            continue
        else:
            re +=s[i]
    return re


In [3]:
# 爬取章节内容
def get_chapter(url):
    html = requests.get(url).content.decode('gbk')

    title = re.search('<title>我在惊悚游戏里封神（无限）\|壶鱼辣椒\|正文  (.*?)章',html,re.S).group(1)
    print(title)
    content = re.findall('class="read-content" id="chaptercontent">(.*?)</div>',html,re.S)
    # 清空乱码
    if content == []:
        print("搜索不到正文的url=",url)
        return ""
    s = content[0].replace('\n','').replace('&nbsp;','').replace('<br/>','\n')
    return title,clearQ(s)

In [4]:
# 判断是否有下一页
def ifnext(s):
    n = re.search('<span class="right"><a class="onclick" href="(.*?)">下一页</a></span>',s,re.S)
    return n
   

In [5]:
# 爬取目录
def get_mulu(html):
    mululist = re.findall('<a href="(.*?)">(.*?)</a>',html,re.S)
    return mululist

In [6]:
# 获取所有的章节标题和链接
def get(url,true_mulu,titles):
    html = requests.get(url).content.decode('gbk')
    mulu = re.search('<div class="booklist">(.*?)<ul>(.*?)</ul>',html,re.S)
    res = ifnext(mulu.group(1))
    if res != None:
        tmp = 'https://www.jjxsxs.com'+res.group(1).strip("'")
        get(tmp,true_mulu,titles)
    
    mululist = get_mulu(mulu.group(2))
    
    for index in mululist:
        nurl = 'https://www.jjxsxs.com'+index[0].strip("'")
        titles.append(index[1])
        true_mulu.append(nurl)
        



        


In [7]:
def save(titles,contents,name):
    os.makedirs(name,exist_ok=True)
    for i in range(len(titles)):
        with open(os.path.join(name,titles[i]+'.txt'),'w',encoding='utf-8') as f:
            f.write(contents[i])


In [8]:
titles = []
contents = []
get('https://www.jjxsxs.com/11/11536/',contents,titles)



In [9]:
pool = Pool(5)
step = 20
res = []
for i in range(0,len(contents),step):
  
    end = min(i+step,len(contents))
    print("正在下载章节",i+1,"~",end+1)
    tmp = pool.map(get_chapter,contents[i:end])
    time.sleep(6)
    res += tmp


正在下载章节 1 ~ 21
第503
第502
第504
第505
第501
第500
第507
第506
第510
第509
第508
第512
第514
第511
第513
第517
第516
第519
第515
第518
正在下载章节 21 ~ 41
第521
第520
第523
第524
第525
第526
第527
第528
第529
第522
第530
第531
第532
第533
第536
第534
第537
第538
第539
第535
正在下载章节 41 ~ 61
第542
第544
第540
第543
第541
第545
第547
第546
第550
第548
第551
第552
第555
第554
第553
第556
第557
第558
第559
第549
正在下载章节 61 ~ 81
第562
第564
第563
第560
第561
第565
第569
第568
第566
第570
第567
第571
第572
第573
第574
第575
第577
第579
第578
第576
正在下载章节 81 ~ 101
第584
第580
第583
第581
第585
第586
第582
第587
第591
第590
第589
第592
第594
第588
第400
第402
第403
第404
第401
第593
正在下载章节 101 ~ 121
第407
第405
第406
第408
第411
第409
第412
第410
第413
第416
第414
第415
第417
第418
第419
第422
第421
第420
第424
第423
正在下载章节 121 ~ 141
第429
第428
第425
第426
第427
第430
第431
第432
第434
第435
第438
第437
第439
第440
第436
第433
第442
第444
第441
第443
正在下载章节 141 ~ 161
第446
第447
第449
第448
第450
第445
第453
第454
第451
第456
第452
第457
第455
第458
第460
第461
第462
第464
第463
第459
正在下载章节 161 ~ 181
第467
第465
第466
第470
第469
第472
第468
第473
第475
第476
第471
第4

In [12]:
name = '我在惊悚游戏里封神'
path = '/home/aistudio/我在惊悚游戏里封神'
os.makedirs(name,exist_ok=True)
for i in range(len(res)):
    with open(os.path.join(path,res[i][0]+'章.txt'),'w',encoding='utf-8') as f:
        f.write(res[i][1])